**Data cleaning part**(Done based on logic)

In [2]:

#remove unnessecery data and fill the null ones
from asyncio.windows_events import NULL
from cmath import nan
import pandas as pd
import category_encoders as ce 
from sklearn.preprocessing import LabelEncoder


def clean(Loan_data):
    Loan_data.drop('Loan_ID',axis=1,inplace=True)

    #filled null values based on the value of the Loan_Status :
    #If yes then:-
    #Self_Employed is Yes and Dependents is min(0) and Married is No and vise versa
    # rest of the values filled with the mean (but integer values not float) 

    Loan_data['Gender']=Loan_data['Gender'].fillna('Female')
    Loan_data['LoanAmount']=Loan_data['LoanAmount'].fillna(146)
    Loan_data['Loan_Amount_Term']=Loan_data['Loan_Amount_Term'].fillna(360)
    for indx,i in enumerate(Loan_data['Credit_History']):
        if(pd.isna(i)):
            if(Loan_data['Loan_Status'][indx]=='N'):Loan_data['Credit_History'][indx]=0
            else:Loan_data['Credit_History'][indx]=1

    for indx,i in enumerate(Loan_data['Self_Employed']):
        if(pd.isna(i)):
            if(Loan_data['Loan_Status'][indx]=='N'):Loan_data['Self_Employed'][indx]='No'
            else:Loan_data['Self_Employed'][indx]='Yes'

    for indx,i in enumerate(Loan_data['Dependents']):
        if(pd.isna(i)):
            if(Loan_data['Loan_Status'][indx]=='N'):Loan_data['Dependents'][indx]='3+'
            else:Loan_data['Dependents'][indx]='0'
 
    for indx,i in enumerate(Loan_data['Married']):
        if(pd.isna(i)):
            if(Loan_data['Loan_Status'][indx]=='N'):Loan_data['Married'][indx]='Yes'
            else:Loan_data['Married'][indx]='No'
    
    
    #data labeled
    print(Loan_data)
    encoder = ce.OrdinalEncoder(['Gender','Married','Education','Self_Employed','Property_Area'])
    Loan_data = encoder.fit_transform(Loan_data)

    encoder = LabelEncoder()
    Loan_data['Loan_Status'] = encoder.fit_transform(Loan_data['Loan_Status'])

    print(Loan_data)
    #Loan_data.to_csv("train.csv")
 
    return Loan_data.astype(int)


**Training part**(Bagging,Gradiant boosting and SVC ,all with Grid search cv)

In [21]:
from copyreg import pickle
from sklearn.model_selection import train_test_split,StratifiedShuffleSplit,GridSearchCV
from sklearn.ensemble import BaggingClassifier
import pickle as pk
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

def train_model(Loan_data):

    cols = [0,12]
    X= Loan_data.drop(['Loan_Status'],axis=1)
    y = Loan_data['Loan_Status']

    X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=.20,random_state=21)

    
    #train using bagging classifier Grid search

    # n_estimators = [10,30,50,70,80,150,160, 170,175,180,185];
    # cv = StratifiedShuffleSplit(n_splits=10, test_size=.30, random_state=15)

    # parameters = {'n_estimators':n_estimators,
              
    #     }
    # grid_bagg = GridSearchCV(BaggingClassifier(base_estimator= None, ## If None, then the base estimator is a decision tree.
    #                                   bootstrap_features=False),
    #                              param_grid=parameters,
    #                              cv=cv,
    #                              n_jobs = -1)
    # grid_bagg.fit(X_train,y_train) 
    # pk.dump(grid_bagg, open("Model_BaggingClassifierGridSearchCV", 'wb'))

    # print (grid_bagg.best_score_)
    # print (grid_bagg.best_params_)
    # print (grid_bagg.best_estimator_)
    
    # bagg_grid = grid_bagg.best_estimator_
    # bagg_train_grid = round(bagg_grid.score(X_train, y_train) * 100, 2)
    # bagg_accuracy_grid = bagg_grid.score(X_train,y_train).round(2)*100

    # print("Training Accuracy with GridSearch :",bagg_train_grid  ,"%")
    # print("Model Accuracy with GridSearch    :",bagg_accuracy_grid ,"%")
    # print("\033[1m--------------------------------------------------------\033[0m")


    # GradientBoostingClassifierGridSearchCV

#     param_grid = {
#         'learning_rate': [0.01, 0.025, 0.05, 0.075, 0.1],
#         'n_estimators':[140, 150, 160], 
#         'max_depth':[3, 4, 5]       
#         }

#     cv = StratifiedShuffleSplit(n_splits=10, test_size=.30, random_state=15)
# # The grid search object
#     grid_gbc = GridSearchCV(GradientBoostingClassifier(random_state=42), 
#                       param_grid=param_grid, 
#                       scoring='f1',
#                       cv = cv,
#                       verbose=0, 
#                       n_jobs=-1)

#     grid_gbc.fit(X_train,y_train) 
#     pk.dump(grid_gbc, open("GradientBoostingClassifierGridSearchCV", 'wb'))

#     gbc_grid = grid_gbc.best_estimator_

#     gbc_train_grid = round(gbc_grid.score(X_train, y_train) * 100, 2)
#     gbc_accuracy_grid = gbc_grid.score(X_train,y_train).round(2)*100
#     print (grid_gbc.best_score_)
#     print (grid_gbc.best_params_)
#     print (grid_gbc.best_estimator_)
#     print("Training Accuracy with GridSearch :",gbc_train_grid  ,"%")
#     print("Model Accuracy with GridSearch    :",gbc_accuracy_grid ,"%")
#     print("\033[1m--------------------------------------------------------\033[0m")

    # SVC GridSearchCV 📊📈
    param_grid = {'C': [0.1, 1, 10, 100, 1000],
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['rbf']} # 'precomputed','sigmoid','linear','pol

    cv = StratifiedShuffleSplit(n_splits=10, test_size=.30, random_state=15)
    grid_svc = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3,cv=cv)
 
    # fitting the model for grid search
    grid_svc.fit(X_train, y_train)
    pk.dump(grid_svc, open("Model_SVCGridSearchCV", 'wb'))

    GridSearchCV(cv=StratifiedShuffleSplit(n_splits=10, random_state=15, test_size=0.3,
            train_size=None),
             estimator=SVC(),
             param_grid={'C': [0.1, 1, 10, 100, 1000],
                         'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
                         'kernel': ['rbf']},
             verbose=3)
    

    print (grid_svc.best_score_)
    print (grid_svc.best_params_)
    print (grid_svc.best_estimator_)

    ### Using the best parameters from the grid-search.
    svc_grid = grid_svc.best_estimator_

    svc_train_grid = round(svc_grid.score(X_train, y_train) * 100, 2)
    svc_accuracy_grid = round(svc_grid.score(X_train,y_train), 2) * 100

    print("Training Accuracy with GridSearch :",svc_train_grid  ,"%")
    print("Model Accuracy with GridSearch    :",svc_accuracy_grid ,"%")
    print("\033[1m--------------------------------------------------------\033[0m")

    pred = svc_grid.predict(X_test)
    return pred, y_test

    

In [4]:
train = pd.read_csv("data/train_u6lujuX_CVtuZ9i.csv")
test = pd.read_csv("data/test_Y3wMUE5_7gLdaTN.csv")

In [5]:
loan = clean(train)

     Gender Married Dependents     Education Self_Employed  ApplicantIncome  \
0      Male      No          0      Graduate            No             5849   
1      Male     Yes          1      Graduate            No             4583   
2      Male     Yes          0      Graduate           Yes             3000   
3      Male     Yes          0  Not Graduate            No             2583   
4      Male      No          0      Graduate            No             6000   
..      ...     ...        ...           ...           ...              ...   
609  Female      No          0      Graduate            No             2900   
610    Male     Yes         3+      Graduate            No             4106   
611    Male     Yes          1      Graduate            No             8072   
612    Male     Yes          2      Graduate            No             7583   
613  Female      No          0      Graduate           Yes             4583   

     CoapplicantIncome  LoanAmount  Loan_Amount_Ter

C:\Users\User\AppData\Local\Temp\ipykernel_45648\652218323.py:23: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  else:Loan_data['Credit_History'][indx]=1
C:\Users\User\AppData\Local\Temp\ipykernel_45648\652218323.py:23: SettingWithCopyWarning

In [22]:
pred,y_test = train_model(loan)

Fitting 10 folds for each of 25 candidates, totalling 250 fits
[CV 1/10] END .......C=0.1, gamma=1, kernel=rbf;, score=0.689 total time=   0.0s
[CV 2/10] END .......C=0.1, gamma=1, kernel=rbf;, score=0.689 total time=   0.0s
[CV 3/10] END .......C=0.1, gamma=1, kernel=rbf;, score=0.689 total time=   0.0s
[CV 4/10] END .......C=0.1, gamma=1, kernel=rbf;, score=0.689 total time=   0.0s
[CV 5/10] END .......C=0.1, gamma=1, kernel=rbf;, score=0.689 total time=   0.0s
[CV 6/10] END .......C=0.1, gamma=1, kernel=rbf;, score=0.689 total time=   0.0s
[CV 7/10] END .......C=0.1, gamma=1, kernel=rbf;, score=0.689 total time=   0.0s
[CV 8/10] END .......C=0.1, gamma=1, kernel=rbf;, score=0.689 total time=   0.0s
[CV 9/10] END .......C=0.1, gamma=1, kernel=rbf;, score=0.689 total time=   0.0s
[CV 10/10] END ......C=0.1, gamma=1, kernel=rbf;, score=0.689 total time=   0.0s
[CV 1/10] END .....C=0.1, gamma=0.1, kernel=rbf;, score=0.689 total time=   0.0s
[CV 2/10] END .....C=0.1, gamma=0.1, kernel=rb

In [14]:
loan.columns

Index(['Gender', 'Married', 'Dependents', 'Education', 'Self_Employed',
       'ApplicantIncome', 'CoapplicantIncome', 'LoanAmount',
       'Loan_Amount_Term', 'Credit_History', 'Property_Area', 'Loan_Status'],
      dtype='object')

In [26]:
y_test,pred

(452    1
 391    0
 529    0
 594    0
 448    1
       ..
 313    0
 458    0
 130    0
 429    0
 457    1
 Name: Loan_Status, Length: 123, dtype: int32,
 array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]))

In [27]:
from sklearn.metrics import accuracy_score

accuracy_score(pred, y_test)

0.6666666666666666